In [ ]:
#This one is the connection to BigQuery and collection to events_ table (daily not interdate) to get events from the last 7 days in order to create the 
# source of truth that can be used as the reference for further comparison

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-auth-oauthlib


In [ ]:
import os, sys, pathlib
print("Kernel Python:", sys.executable)
# 1) Avoid the default cache (moves it to a safe ASCII path)
os.environ["PIP_CACHE_DIR"] = r"C:\pipcache"

# 2) Upgrade pip tooling in THIS kernel
!{sys.executable} -m pip install --upgrade pip setuptools wheel

# 3) Install your packages without using the cache (prevents file lock issues)
%pip install --no-cache-dir google-cloud-bigquery PyYAML requests


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-cloud-bigquery PyYAML requests


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
import sys
!{sys.executable} -m pip install pandas pyarrow db-dtypes

In [ ]:
# Install (only once per environment)
# !pip install google-auth google-auth-oauthlib google-cloud-bigquery

# BigQuery connection
# 1️⃣ Your GCP “App” project
# This is the one you used to create the client_secret.json file. It’s the project where you:
# - Created the OAuth consent screen
# - Created the OAuth Client ID (Desktop App)
# - Possibly store your own scripts or Cloud Functions
# This project is used only for authentication and API access — it doesn’t need to be the same one that contains the GA4 data. It provides the credentials you use in your notebook.

# 2️⃣ The GA4 BigQuery export project
# This is a separate project, automatically created or linked by Google Analytics when you set up the BigQuery export in GA4 Admin → BigQuery Linking.
# This project contains:
# - The GA4 dataset (usually named analytics_<propertyid>)
# - Tables like events_20251110, events_intraday_20251111, etc.
# This project is where your data lives, and it’s the one you need read access to.

import os, json
from google.cloud import bigquery
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# ----- EDIT THESE -----
CLIENT_SECRET_FILE = r"YOURPATH\client_secret.json"
PROJECT_ID = "YOUR-PROJECT-ID"
SCOPES = [
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/cloud-platform",
]
TOKEN_PATH = "token.json"
# ----------------------

SCOPES = ["https://www.googleapis.com/auth/bigquery"]
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
creds = flow.run_local_server(port=0)

# Note: project here is the GA4 one, not the OAuth one
client = bigquery.Client(project="YOUR-PROJECT-ID", credentials=creds)

for d in client.list_datasets():
    print(" -", d.dataset_id)

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
import re, datetime as dt

# --- set these two correctly ---
BQ_PROJECT = "YOUR-PROJECT-ID"           # GA4 export project 
BQ_DATASET = "YOUR-DATASET"        # GA4 dataset name from GA4 Admin → BigQuery Linking 
BQ_TABLE_WC  = "events_*"                  # GA4 export table wildcard
BQ_LOCATION  = "EU"                        # "EU" or "US" (match your dataset)
# --------------------------------

# 1) Discover dataset location (EU/US) - got some issue if sending the location straight
meta_client = bigquery.Client(project=BQ_PROJECT, credentials=creds)  # no location here
ds = meta_client.get_dataset(f"{BQ_PROJECT}.{BQ_DATASET}")            # raises NotFound if typo
DATASET_LOCATION = ds.location
print("Dataset location:", DATASET_LOCATION)

# 2) Create a client bound to the correct location
client = bigquery.Client(project=BQ_PROJECT, credentials=creds, location=DATASET_LOCATION)

# 3) Find the newest events table (daily first, else intraday)
tables = list(client.list_tables(f"{BQ_PROJECT}.{BQ_DATASET}"))
latest_daily, latest_intra = None, None
for t in tables:
    if re.match(r"events_\d{8}$", t.table_id):
        latest_daily = max(latest_daily or t.table_id, t.table_id)
    elif re.match(r"events_intraday_\d{8}$", t.table_id):
        latest_intra = max(latest_intra or t.table_id, t.table_id)

latest = latest_daily or latest_intra
print("Latest table:", latest)
assert latest, "No GA4 events tables found. Check link/export or permissions."

# 4) Quick check: Preview a few rows from that specific table
sql = f"""
SELECT event_name, COUNT(*) AS c
FROM `{BQ_PROJECT}.{BQ_DATASET}.{latest}`
GROUP BY event_name
ORDER BY c DESC
LIMIT 10
"""
# run your query
job = client.query(sql)  # client already created with correct location
rows = list(job.result())





In [ ]:
import re
from datetime import datetime
from google.cloud import bigquery

# assumes you already have: creds, BQ_PROJECT, BQ_DATASET, DATASET_LOCATION (europe-north1)
meta_client = bigquery.Client(project=BQ_PROJECT, credentials=creds)
ds = meta_client.get_dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
DATASET_LOCATION = ds.location  # keep using europe-north1

client = bigquery.Client(project=BQ_PROJECT, credentials=creds, location=DATASET_LOCATION)

def newest_tables(client, project, dataset, max_tables=3):
    tables = list(client.list_tables(f"{project}.{dataset}"))
    parsed = []
    for t in tables:
        tid = t.table_id
        m = re.match(r"(events|events_intraday)_(\d{8})$", tid)
        if m:
            parsed.append((tid, datetime.strptime(m.group(2), "%Y%m%d"), m.group(1)))
    # sort by date desc and pick a few
    parsed.sort(key=lambda x: x[1], reverse=True)
    return [tid for (tid, _, _) in parsed[:max_tables]]

latest_tbls = newest_tables(client, BQ_PROJECT, BQ_DATASET, max_tables=3)
print("Using tables:", latest_tbls)
assert latest_tbls, "No GA4 events tables found."


In [ ]:
import yaml, math, re
# PII CHECK FOR PARAMETERS
# ✅ raw strings for regex to avoid SyntaxWarning
PII_KEY_RE = re.compile(r"(email|name|phone|address|login|ssn|iban)", re.I)
PII_VAL_RE = re.compile(r"@|(\+\d{6,})|[0-9]{8,}")

def generate_autospec_fast(client, project, dataset, tables, sample_rate=0.002, out_path="tracking_spec.generated.yml"):
    # Build a UNION over latest tables
    union_sql = " \nUNION ALL\n".join([f"SELECT event_name, event_params, items FROM `{project}.{dataset}.{t}`" for t in tables])

    # ✅ triple quotes properly closed, BOOL_OR replaced with COUNTIF(...) > 0
    sql = f"""
    WITH unioned AS (
      {union_sql}
    ),
    sampled AS (
      SELECT * FROM unioned
      WHERE RAND() < {sample_rate}
      LIMIT 200000
    ),
    flat AS (
      SELECT
        event_name,
        ep.key AS param_key,
        COALESCE(
          ep.value.string_value,
          CAST(ep.value.int_value AS STRING),
          CAST(ep.value.double_value AS STRING)
        ) AS param_val,
        (items IS NOT NULL AND ARRAY_LENGTH(items) > 0) AS items_present,
        SAFE_CAST(COALESCE(
          ep.value.string_value,
          CAST(ep.value.int_value AS STRING),
          CAST(ep.value.double_value AS STRING)
        ) AS FLOAT64) AS num_val
      FROM sampled, UNNEST(event_params) ep
    ),
    stats AS (
      SELECT
        event_name,
        param_key,
        COUNT(*) AS rows_total,
        COUNTIF(param_val IS NOT NULL AND param_val != '') AS rows_present,
        COUNTIF(num_val IS NOT NULL) AS num_count,
        MIN(num_val) AS min_num,
        ARRAY_AGG(DISTINCT param_val IGNORE NULLS LIMIT 50) AS samples,
        COUNTIF(items_present) > 0 AS items_present_any
      FROM flat
      GROUP BY 1,2
    )
    SELECT * FROM stats
    ORDER BY event_name, param_key
    """
    rows = client.query(sql).result()
    summary = [dict(r) for r in rows]

    # Build YAML
    out = {"meta": {"spec_version": "auto-1.0"}, "events": []}
    req_threshold = 0.98

    from collections import defaultdict
    by_event = defaultdict(list)
    items_presence = defaultdict(bool)

    for r in summary:
        ev = r["event_name"]
        k = r["param_key"]
        rows_total = r["rows_total"] or 1
        rows_present = r["rows_present"] or 0
        presence_rate = rows_present / rows_total
        num_count = r["num_count"] or 0
        min_num = r["min_num"]
        samples = r["samples"] or []
        items_presence[ev] = items_presence[ev] or bool(r["items_present_any"])

        # skip obvious PII
        if PII_KEY_RE.search(k or "") or any(PII_VAL_RE.search((s or "")) for s in samples[:20]):
            continue

        # type inference
        if num_count >= max(5, math.ceil(0.5 * len(samples))):
            ptype = "integer" if (min_num is not None and float(min_num).is_integer()) else "number"
        else:
            ptype = "string"

        enum = None
        if ptype == "string" and 0 < len(samples) <= 20 and presence_rate >= 0.95:
            enum = sorted(samples)

        entry = {"key": k, "type": ptype, "required": presence_rate >= req_threshold}
        if ptype in ("number", "integer") and min_num is not None and min_num >= 0:
            entry["min"] = 0
        if enum:
            entry["enum"] = enum

        by_event[ev].append(entry)

    for ev, params in by_event.items():
        ev_entry = {"name": ev, "required_params": params}
        if items_presence[ev]:
            ev_entry["required_params"].append({
                "key": "items", "type": "array<object>", "required": True,
                "item_schema": [
                    {"key": "item_id", "type": "string", "required": True},
                    {"key": "quantity", "type": "integer", "required": True, "min": 1},
                    {"key": "price", "type": "number", "required": False, "min": 0}
                ]
            })
        out["events"].append(ev_entry)

    # ✅ raw strings used in regex-like text patterns
    out["consent"] = {"required_storages": ["analytics_storage", "ad_storage"], "marketing_vendors_block_when_denied": True}
    out["pii_policy"] = {
        "disallow_param_keys": ["email", "phone", "first_name", "last_name", "address"],
        "text_patterns": [r"(?i)[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}", r"(?i)\b(?:\+?\d{1,3}[\s-]?)?(?:\d[\s-]?){8,}\b"]
    }

    with open(out_path, "w", encoding="utf-8") as f:
        yaml.safe_dump(out, f, sort_keys=False, allow_unicode=True)
    return out_path



In [ ]:
#detect-only URL-param PII scanner
import yaml, math, re

# ----------------------------
# PII CHECK FOR PARAMETERS
# ----------------------------
EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b",re.IGNORECASE,)
PHONE_RE = re.compile(
    r"""
    \b
    (?:\+|00)\d            # +358 / 0044 etc.
    [\d\-\s]{5,16}         # allow spaces/dashes but total length reasonable
    \b
    """,
    re.VERBOSE,
)
LONG_NUMERIC_RE = re.compile(r"\b\d{9,}\b")
IPV4_RE = re.compile(r"\b(?:\d{1,3}\.){3}\d{1,3}\b")
IPV6_RE = re.compile(r"\b(?:[0-9a-f]{1,4}:){2,7}[0-9a-f]{1,4}\b", re.IGNORECASE,)
CREDITCARD_RE = re.compile(r"\b(?:\d[ -]*?){13,19}\b")
FINNISH_HETU_RE = re.compile(r"\b\d{6}[-+A]\d{3}[0-9A-FHJ-NPR-Y]\b", re.IGNORECASE,)
PII_VALUE_PATTERNS = [
    EMAIL_RE,
    PHONE_RE,
    IPV4_RE,
    IPV6_RE,
    CREDITCARD_RE,
    FINNISH_HETU_RE,
    LONG_NUMERIC_RE,
]
PII_VALUE_PATTERNS_LOOSE = [EMAIL_RE, PHONE_RE, IPV4_RE, IPV6_RE]
PII_VALUE_PATTERNS_STRICT = PII_VALUE_PATTERNS_LOOSE + [CREDITCARD_RE, FINNISH_HETU_RE, LONG_NUMERIC_RE]

def looks_like_pii_value(value: str) -> bool:
    if not value:
        return False
    v = str(value)
    return any(p.search(v) for p in PII_VALUE_PATTERNS_STRICT)


PII_KEY_RE = re.compile(r"(email|name|phone|address|login|ssn|iban)", re.I)
PII_VAL_RE = re.compile(r"@|(\+\d{6,})|[0-9]{8,}")

# Which GA4 params commonly carry full URLs
URL_PARAM_KEYS = ("page_location", "page_referrer", "link_url")


def generate_autospec_fast(
    client,
    project,
    dataset,
    tables,
    sample_rate=0.002,
    out_path="tracking_spec.generated.yml",
):
    """
    Builds a sampled inventory of GA4 event parameters, infers simple types, skips obvious PII,
    and (NEW) detects PII in URL query parameters (detect-only). Outputs a YAML spec plus
    a 'pii_url_findings' section for easy review / alerting.

    Args:
        client: google.cloud.bigquery.Client
        project: GCP project id
        dataset: BigQuery dataset id for GA4 export
        tables: list[str] of table ids to UNION (e.g., last N days)
        sample_rate: float (0..1), probabilistic sampling per row
        out_path: path to write YAML spec
    """
    # ----------------------------
    # 1) Parameter stats (original logic)
    # ----------------------------
    union_sql = " \nUNION ALL\n".join(
        [f"SELECT event_name, event_params, items FROM `{project}.{dataset}.{t}`" for t in tables]
    )

    # NOTE: This is a normal f-string. The SQL here doesn't use backslash-heavy regex,
    # so we don't need the raw prefix (r or rf). Where we *do* use regex in SQL later,
    # we use an rf"""...""" string and double braces {{...}} for quantifiers.
    sql = f"""
    WITH unioned AS (
      {union_sql}
    ),
    sampled AS (
      SELECT * FROM unioned
      WHERE RAND() < {sample_rate}
      LIMIT 200000
    ),
    flat AS (
      SELECT
        event_name,
        ep.key AS param_key,
        COALESCE(
          ep.value.string_value,
          CAST(ep.value.int_value AS STRING),
          CAST(ep.value.double_value AS STRING)
        ) AS param_val,
        (items IS NOT NULL AND ARRAY_LENGTH(items) > 0) AS items_present,
        SAFE_CAST(COALESCE(
          ep.value.string_value,
          CAST(ep.value.int_value AS STRING),
          CAST(ep.value.double_value AS STRING)
        ) AS FLOAT64) AS num_val
      FROM sampled, UNNEST(event_params) ep
    ),
    stats AS (
      SELECT
        event_name,
        param_key,
        COUNT(*) AS rows_total,
        COUNTIF(param_val IS NOT NULL AND param_val != '') AS rows_present,
        COUNTIF(num_val IS NOT NULL) AS num_count,
        MIN(num_val) AS min_num,
        ARRAY_AGG(DISTINCT param_val IGNORE NULLS LIMIT 50) AS samples,
        COUNTIF(items_present) > 0 AS items_present_any
      FROM flat
      GROUP BY 1,2
    )
    SELECT * FROM stats
    ORDER BY event_name, param_key
    """
    rows = client.query(sql).result()
    summary = [dict(r) for r in rows]

    # ----------------------------
    # 2) URL-parameter PII detector (detect-only)
    # ----------------------------
    # Use a *raw f-string* so backslashes in regex aren't treated as Python escapes.
    # IMPORTANT: In f-strings, '{' and '}' are special, so we double them for regex quantifiers: {{2,}}, {{7,}}, etc.
    url_pii_sql = rf"""
    WITH unioned AS (
      {union_sql}
    ),
    sampled AS (
      SELECT * FROM unioned
      WHERE RAND() < {sample_rate}
      LIMIT 200000
    ),
    urls AS (
      SELECT
        event_name,
        ep.key AS param_key,
        ep.value.string_value AS url_val
      FROM sampled, UNNEST(event_params) ep
      WHERE ep.key IN ('page_location','page_referrer','link_url')
        AND ep.value.string_value IS NOT NULL
        AND ep.value.string_value LIKE '%?%'
    ),
    pairs AS (
      SELECT
        event_name, param_key, url_val,
        SPLIT(url_val, '?')[SAFE_OFFSET(1)] AS query_str
      FROM urls
    ),
    kv AS (
      SELECT
        event_name, param_key, url_val,
        kv AS pair,
        LOWER(SPLIT(kv, '=')[SAFE_OFFSET(0)]) AS k,
        SPLIT(kv, '=')[SAFE_OFFSET(1)] AS v_raw
      FROM pairs, UNNEST(SPLIT(query_str, '&')) AS kv
    ),
    norm AS (
      SELECT
        event_name, param_key, url_val, k,
        SAFE.SUBSTR(TRIM(v_raw), 0, 2048) AS v,
        REPLACE(TRIM(v_raw), '%40', '@') AS v_at
      FROM kv
      WHERE v_raw IS NOT NULL
    ),
    matches AS (
      SELECT
        event_name,
        param_key,
        k AS query_param,
        CASE
          WHEN REGEXP_CONTAINS(LOWER(v_at),
               r'(^|[^a-z0-9._%+\-])[a-z0-9._%+\-]+@[^ \s/?#&]+\.[a-z]{{2,}}([^a-z]|$)')
            THEN 'email'
          WHEN REGEXP_CONTAINS(v, r'(^|\D)(\+?\d[\d\-\s() ]{{7,}})(\D|$)')
            THEN 'phone'
          WHEN REGEXP_CONTAINS(LOWER(k), r'(email|e-?mail|user(name)?|login|phone|mobile|msisdn|ssn|nid|token|auth|session|address)')
            THEN 'suspicious_key'
          WHEN REGEXP_CONTAINS(v, r'([A-Fa-f0-9]{{16,}}|[A-Za-z0-9_\-]{{24,}})')
            THEN 'id_like'
          ELSE NULL
        END AS pii_type,
        -- Mask local parts of emails and long numbers in the *full* URL for safe sharing
        REGEXP_REPLACE(
          REGEXP_REPLACE(url_val, r'([a-zA-Z0-9._%+\-]+)@', r'***@'),
          r'(\d{{6,}})', '***'
        ) AS sample_url_masked
      FROM norm
      WHERE
        REGEXP_CONTAINS(LOWER(v_at),
          r'(^|[^a-z0-9._%+\-])[a-z0-9._%+\-]+(@|%40)[^ \s/?#&]+\.[a-z]{{2,}}([^a-z]|$)')
        OR REGEXP_CONTAINS(v, r'(^|\D)(\+?\d[\d\-\s() ]{{7,}})(\D|$)')
        OR REGEXP_CONTAINS(LOWER(k), r'(email|e-?mail|user(name)?|login|phone|mobile|msisdn|ssn|nid|token|auth|session|address)')
        OR REGEXP_CONTAINS(v, r'([A-Fa-f0-9]{{16,}}|[A-Za-z0-9_\-]{{24,}})')
    ),
    agg AS (
      SELECT
        event_name,
        param_key,
        query_param,
        pii_type,
        COUNT(*) AS events_flagged,
        ARRAY_AGG(DISTINCT sample_url_masked IGNORE NULLS LIMIT 10) AS sample_urls_masked
      FROM matches
      WHERE pii_type IS NOT NULL
      GROUP BY 1,2,3,4
    )
    SELECT * FROM agg
    ORDER BY events_flagged DESC
    """
    url_rows = client.query(url_pii_sql).result()
    url_findings = [dict(r) for r in url_rows]

    # ----------------------------
    # 3) Build YAML (original logic + new pii_url_findings)
    # ----------------------------
    out = {"meta": {"spec_version": "auto-1.0"}, "events": []}
    req_threshold = 0.98

    from collections import defaultdict

    by_event = defaultdict(list)
    items_presence = defaultdict(bool)

    for r in summary:
        ev = r["event_name"]
        k = r["param_key"]
        rows_total = r["rows_total"] or 1
        rows_present = r["rows_present"] or 0
        presence_rate = rows_present / rows_total
        num_count = r["num_count"] or 0
        min_num = r["min_num"]
        samples = r["samples"] or []
        items_presence[ev] = items_presence[ev] or bool(r["items_present_any"])

        # Skip obvious PII (existing behavior)
        if PII_KEY_RE.search(k or "") or any(
            looks_like_pii_value(s) for s in samples[:20]
        ):
            continue


        # Type inference
        if num_count >= max(5, math.ceil(0.5 * len(samples))):
            ptype = "integer" if (min_num is not None and float(min_num).is_integer()) else "number"
        else:
            ptype = "string"

        enum = None
        if ptype == "string" and 0 < len(samples) <= 20 and presence_rate >= 0.95:
            enum = sorted(samples)

        entry = {"key": k, "type": ptype, "required": presence_rate >= req_threshold}
        if ptype in ("number", "integer") and min_num is not None and min_num >= 0:
            entry["min"] = 0
        if enum:
            entry["enum"] = enum

        by_event[ev].append(entry)

    for ev, params in by_event.items():
        ev_entry = {"name": ev, "required_params": params}
        if items_presence[ev]:
            ev_entry["required_params"].append(
                {
                    "key": "items",
                    "type": "array<object>",
                    "required": True,
                    "item_schema": [
                        {"key": "item_id", "type": "string", "required": True},
                        {"key": "quantity", "type": "integer", "required": True, "min": 1},
                        {"key": "price", "type": "number", "required": False, "min": 0},
                    ],
                }
            )
        out["events"].append(ev_entry)

    # Policy (unchanged)
    out["consent"] = {
        "required_storages": ["analytics_storage", "ad_storage"],
        "marketing_vendors_block_when_denied": True,
    }
    out["pii_policy"] = {
        "disallow_param_keys": ["email", "phone", "first_name", "last_name", "address"],
        "text_patterns": [
            r"(?i)[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}",
            r"(?i)\b(?:\+?\d{1,3}[\s-]?)?(?:\d[\s-]?){8,}\b",
        ],
    }

    # NEW: attach detect-only URL PII summary for BI/Slack
    per_event = defaultdict(list)
    for r in url_findings:
        per_event[r["event_name"]].append(
            {
                "page_param": r["param_key"],
                "query_param": r["query_param"],
                "pii_type": r["pii_type"],
                "events_flagged": int(r["events_flagged"]),
                "sample_urls_masked": r.get("sample_urls_masked", []),
            }
        )
    out["pii_url_findings"] = [{"event": ev, "offenders": offenders} for ev, offenders in per_event.items()]

    with open(out_path, "w", encoding="utf-8") as f:
        yaml.safe_dump(out, f, sort_keys=False, allow_unicode=True)

    return out_path


In [ ]:
yaml_path = generate_autospec_fast(client, BQ_PROJECT, BQ_DATASET, latest_tbls, sample_rate=0.002)
print("✅ Wrote:", yaml_path)

# quick peek
with open(yaml_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i > 60: break
        print(line.rstrip())

In [ ]:
# --- sampled, fast PII audit over latest GA4 tables ---
from google.cloud.bigquery import QueryJobConfig

SAMPLE_RATE = 0.001   # 0.1% sample; raise/lower if needed
MAX_ROWS    = 200000  # safety cap for sampled CTE
BYTES_CAP   = 2 * 1024**3  # 2 GB guardrail; adjust if needed

union_sql = " \nUNION ALL\n".join([
    f"SELECT event_name, event_params, user_properties, items FROM `{BQ_PROJECT}.{BQ_DATASET}.{t}`"
    for t in latest_tbls
])

sql_pii = f"""
WITH base AS (
  {union_sql}
),
sampled AS (
  SELECT * FROM base
  WHERE RAND() < {SAMPLE_RATE}
  LIMIT {MAX_ROWS}
),
-- 1) event_params
ev AS (
  SELECT
    'event_param' AS source,
    event_name,
    ep.key AS param_key,
    COALESCE(ep.value.string_value,
             CAST(ep.value.int_value AS STRING),
             CAST(ep.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(event_params) ep
),
-- 2) user_properties (GA4 structure is similar to event_params)
up AS (
  SELECT
    'user_property' AS source,
    event_name,
    up.key AS param_key,
    COALESCE(up.value.string_value,
             CAST(up.value.int_value AS STRING),
             CAST(up.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(user_properties) up
),
-- 3) item parameters (if any)
ip AS (
  SELECT
    'item_param' AS source,
    event_name,
    ip.key AS param_key,
    COALESCE(ip.value.string_value,
             CAST(ip.value.int_value AS STRING),
             CAST(ip.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(items) it, UNNEST(it.item_params) ip
),
flat AS (
  SELECT * FROM ev
  UNION ALL SELECT * FROM up
  UNION ALL SELECT * FROM ip
)
SELECT
  source,
  event_name,
  param_key,
  COUNT(*) AS hits,
  ARRAY_AGG(DISTINCT SUBSTR(param_value,1,80) IGNORE NULLS LIMIT 5) AS sample_values
FROM flat
WHERE
  -- suspicious keys
  REGEXP_CONTAINS(LOWER(param_key), r'(email|e-mail|mail|name|firstname|lastname|phone|mobile|tel|address|street|zip|postcode|login|user|ssn|iban)')
  -- OR suspicious values: email-like or long digit/phone patterns
  OR REGEXP_CONTAINS(LOWER(param_value), r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{{2,}}')
  OR REGEXP_CONTAINS(param_value, r'(?i)\\b(?:\\+?\\d{{1,3}}[\\s-]?)?(?:\\d[\\s-]? ){{8,}}\\b')
GROUP BY 1,2,3
ORDER BY hits DESC, source, event_name, param_key
"""

job_cfg = QueryJobConfig(maximum_bytes_billed=BYTES_CAP)
pii_rows = list(client.query(sql_pii, job_config=job_cfg).result())
print(f"Rows flagged: {len(pii_rows)}")

# Show a quick preview
for r in pii_rows[:15]:
    print(f"[{r['source']}] {r['event_name']} :: {r['param_key']}  hits={r['hits']}  samples={r['sample_values']}")


In [ ]:
import csv
csv_filename = f"{BQ_DATASET}_pii_audit_results.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["source","event_name","param_key","hits","sample_values"])
    for r in pii_rows:
        w.writerow([r["source"], r["event_name"], r["param_key"], r["hits"], "; ".join(r["sample_values"] or [])])

print("Wrote: pii_audit_results.csv")


In [ ]:
from google.cloud.bigquery import QueryJobConfig

def run_sql(sql, bytes_cap=2 * 1024**3):
    """Executes SQL and prints detailed BigQuery errors if they occur."""
    cfg = QueryJobConfig(maximum_bytes_billed=bytes_cap)
    job = client.query(sql, job_config=cfg)
    try:
        rows = list(job.result())
        return rows
    except Exception as e:
        print("---- BigQuery error details ----")
        try:
            print(job.errors)  # detailed JSON error block
        except Exception:
            pass
        print("Exception:", e)
        raise

def dryrun_sql(sql):
    """Performs a dry run (syntax + cost check, no data scanned)."""
    cfg = QueryJobConfig(dry_run=True, use_query_cache=False)
    job = client.query(sql, job_config=cfg)
    print(f"✅ Query syntax OK — estimated bytes processed: {job.total_bytes_processed:,}")


In [ ]:
SAMPLE_RATE = 0.001
MAX_ROWS = 150000

union_sql = " \nUNION ALL\n".join([
    f"SELECT event_name, event_params FROM `{BQ_PROJECT}.{BQ_DATASET}.{t}`"
    for t in latest_tbls[:1]  # start with just the newest table
])

sql_min = f"""
WITH base AS (
  {union_sql}
),
sampled AS (
  SELECT * FROM base
  WHERE RAND() < {SAMPLE_RATE}
  LIMIT {MAX_ROWS}
),
flat AS (
  SELECT
    event_name,
    ep.key AS param_key,
    COALESCE(ep.value.string_value,
             CAST(ep.value.int_value AS STRING),
             CAST(ep.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(event_params) ep
),
flags AS (
  SELECT
    event_name, param_key, param_value,
    REGEXP_CONTAINS(LOWER(param_key),
      r'(email|e-mail|mail|name|firstname|first_name|lastname|last_name|phone|mobile|tel|address|street|zip|postcode|iban|ssn|nationalid|hetu)'
    ) AS key_suspicious,
    REGEXP_CONTAINS(LOWER(param_value),
      r'^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{{2,}}$'
    ) AS looks_email
  FROM flat
)
SELECT
  'event_param' AS source,
  event_name, param_key,
  COUNT(*) AS hits,
  ARRAY_AGG(DISTINCT SUBSTR(param_value,1,120) IGNORE NULLS LIMIT 5) AS sample_values
FROM flags
WHERE key_suspicious OR looks_email
GROUP BY 1,2,3
ORDER BY hits DESC, event_name, param_key
"""

dryrun_sql(sql_min)
rows_min = run_sql(sql_min)
print("Rows flagged (minimal):", len(rows_min))
rows_min[:10]


In [ ]:
union_sql = " \nUNION ALL\n".join([
    f"""SELECT
           event_name, event_params, user_properties, items
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{t}`"""
    for t in latest_tbls[:1]
])

sql_more = f"""
WITH base AS (
  {union_sql}
),
sampled AS (
  SELECT * FROM base
  WHERE RAND() < {SAMPLE_RATE}
  LIMIT {MAX_ROWS}
),
ev AS (
  SELECT 'event_param' AS source, event_name,
         ep.key AS param_key,
         COALESCE(ep.value.string_value,
                  CAST(ep.value.int_value AS STRING),
                  CAST(ep.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(event_params) ep
),
up AS (
  SELECT 'user_property' AS source, event_name,
         up.key AS param_key,
         COALESCE(up.value.string_value,
                  CAST(up.value.int_value AS STRING),
                  CAST(up.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(user_properties) up
),
ip AS (
  SELECT 'item_param' AS source, event_name,
         ip.key AS param_key,
         COALESCE(ip.value.string_value,
                  CAST(ip.value.int_value AS STRING),
                  CAST(ip.value.double_value AS STRING)) AS param_value
  FROM sampled, UNNEST(items) it, UNNEST(it.item_params) ip
),
flat AS (
  SELECT * FROM ev
  UNION ALL SELECT * FROM up
  UNION ALL SELECT * FROM ip
),
flags AS (
  SELECT
    source, event_name, param_key, param_value,
    REGEXP_CONTAINS(LOWER(param_key),
      r'(email|e-mail|mail|name|firstname|first_name|lastname|last_name|phone|mobile|tel|address|street|zip|postcode|login|user|iban|ssn|nationalid|hetu)'
    ) AS key_suspicious,
    REGEXP_CONTAINS(LOWER(param_value),
      r'^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{{2,}}$'
    ) AS looks_email,
    REGEXP_CONTAINS(param_value,
      r'(?i)\\b(?:\\+?\\d{{1,3}}[\\s-]?)?(?:\\d[\\s-]?){8,}\\b'
    ) AS looks_phone_digits,
    REGEXP_CONTAINS(UPPER(param_value),
      r'\\b[A-Z]{{2}}\\d{{2}}[A-Z0-9]{{11,30}}\\b'
    ) AS looks_iban_generic,
    REGEXP_CONTAINS(UPPER(param_value),
      r'\\bFI\\d{{16}}\\b'
    ) AS looks_iban_fi
  FROM flat
)
SELECT
  source, event_name, param_key,
  COUNT(*) AS hits,
  ARRAY_AGG(DISTINCT SUBSTR(param_value,1,120) IGNORE NULLS LIMIT 5) AS sample_values
FROM flags
WHERE key_suspicious OR looks_email OR looks_phone_digits OR looks_iban_generic OR looks_iban_fi
GROUP BY 1,2,3
ORDER BY hits DESC, source, event_name, param_key
"""

dryrun_sql(sql_more)
rows_more = run_sql(sql_more)
print("Rows flagged (expanded):", len(rows_more))
rows_more[:10]


In [ ]:
# pick whichever result list you want to export
results = ext_rows if 'ext_rows' in locals() else (
    rows_more if 'rows_more' in locals() else (
        rows_min if 'rows_min' in locals() else pii_rows
    )
)
results[:10]

In [ ]:
import csv
csv_file = f"{BQ_DATASET}_pii_audit_results.csv"

with open(csv_file, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow([
        "source","event_name","param_key","hits",
        "by_key","val_email","val_phone","val_ipv4","val_iban","val_credit_card","val_hetu",
        "sample_values"
    ])
    for r in results:
        w.writerow([
            r.get("source"),
            r.get("event_name"),
            r.get("param_key"),
            r.get("hits"),
            r.get("by_key"),
            r.get("val_email"),
            r.get("val_phone"),
            r.get("val_ipv4"),
            r.get("val_iban"),
            r.get("val_credit_card"),
            r.get("val_hetu"),
            "; ".join(r.get("sample_values") or [])
        ])

print(f"✅ Wrote: {csv_file}")



In [ ]:
# -----------------------------
# Integrate anomaly detection (less sensitive)
# -----------------------------
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google.cloud.bigquery import QueryJobConfig

# --- Tunables you can tweak ---
LOOKBACK_DAYS   = 180        # longer baseline
BYTES_CAP       = 2 * 1024**3
TZ              = "Europe/Helsinki"  # for date bucketing in BQ
# global guards
MIN_COUNT_DAY   = 200        # require enough volume on the day
REL_MIN_CHANGE  = 0.12       # require >= 12% relative deviation from baseline
PERSIST_CONSEC  = 2          # require >= 2 consecutive days (set to 1 to disable)

end_date   = datetime.utcnow().date()
start_date = end_date - timedelta(days=LOOKBACK_DAYS)

sql_anom = f"""
SELECT
  event_name,
  DATE(TIMESTAMP_MICROS(event_timestamp), "{TZ}") AS event_date,
  COUNT(1) AS event_count
FROM `{BQ_PROJECT}.{BQ_DATASET}.events_*`
WHERE
  _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL {LOOKBACK_DAYS} DAY))
                   AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
GROUP BY event_name, event_date
ORDER BY event_name, event_date
"""

df = client.query(sql_anom, job_config=QueryJobConfig(maximum_bytes_billed=BYTES_CAP)).to_dataframe()
print(df.head())

# -----------------------------
# Detectors (more conservative)
# -----------------------------
def robust_z_anoms(df, win=28, k=3.5, min_points=14, min_count=MIN_COUNT_DAY, rel_min=REL_MIN_CHANGE):
    """Median/MAD baseline with volume & effect-size guards."""
    df = df.copy()
    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")
    out = []
    for ev, g in df.groupby("event_name", sort=False):
        g = g.sort_values("event_date").copy()
        if len(g) < min_points:
            continue
        # rolling robust baseline
        g["median"] = g["event_count"].rolling(win, min_periods=win//2).median()
        mad = g["event_count"].rolling(win, min_periods=win//2).apply(
            lambda x: np.median(np.abs(x - np.median(x))), raw=False
        )
        # sigma from MAD (avoid zero/NaN)
        sigma = 1.4826 * mad.replace(0, np.nan)
        g["robust_z"] = (g["event_count"] - g["median"]) / sigma
        g["rel_delta"] = (g["event_count"] - g["median"]) / g["median"].replace(0, np.nan)
        g["anomaly_robustz_base"] = (
            g["event_count"] >= min_count
        ) & (
            g["robust_z"].abs() > k
        ) & (
            g["rel_delta"].abs() >= rel_min
        )
        out.append(g[["event_name","event_date","event_count","median","robust_z","rel_delta","anomaly_robustz_base"]])
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

def weekday_robust_anoms(df, lookback_weeks=12, k=3.0, min_points=21, min_count=MIN_COUNT_DAY, rel_min=REL_MIN_CHANGE):
    """Same-DOW robust baseline (median + MAD) over past N weeks."""
    df = df.copy()
    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")
    df["dow"] = df["event_date"].dt.dayofweek
    out = []
    for ev, g in df.groupby("event_name", sort=False):
        g = g.sort_values("event_date").copy()
        if len(g) < min_points:
            continue
        z_vals = []
        rels = []
        meds = []
        for i in range(len(g)):
            cur = g.iloc[i]
            start = cur["event_date"] - pd.Timedelta(days=7*lookback_weeks)
            hist = g.loc[(g["dow"] == cur["dow"]) & (g["event_date"] < cur["event_date"]) & (g["event_date"] >= start), "event_count"]
            if len(hist) >= 6:
                med = np.median(hist)
                mad = np.median(np.abs(hist - med))
                sigma = max(1e-6, 1.4826 * mad)
                z = (cur["event_count"] - med) / sigma
                rel = (cur["event_count"] - med) / (med if med else np.nan)
            else:
                med = np.nan; z = np.nan; rel = np.nan
            meds.append(med); z_vals.append(z); rels.append(rel)
        g["weekday_med"] = meds
        g["weekday_z"] = z_vals
        g["weekday_rel"] = rels
        g["anomaly_weekday_base"] = (
            (g["event_count"] >= min_count)
            & (g["weekday_z"].abs() > k)
            & (g["weekday_rel"].abs() >= rel_min)
        )
        out.append(g[["event_name","event_date","event_count","weekday_med","weekday_z","weekday_rel","anomaly_weekday_base"]])
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

def poisson_band_anoms(df, win=28, z=2.5, min_points=14, min_count=MIN_COUNT_DAY, rel_min=REL_MIN_CHANGE):
    """Poisson band with simple overdispersion from recent residuals."""
    df = df.copy()
    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")
    out = []
    for ev, g in df.groupby("event_name", sort=False):
        g = g.sort_values("event_date").copy()
        if len(g) < min_points:
            continue
        lam = g["event_count"].rolling(win, min_periods=win//2).mean()
        # overdispersion phi from rolling median(|residual|)/0.6745
        resid = g["event_count"] - lam
        sigma_rob = (resid.rolling(win, min_periods=win//2)
                          .apply(lambda x: np.median(np.abs(x - np.median(x))) * 1.4826, raw=False))
        sd = np.sqrt(np.maximum(lam, 1e-6))
        # take larger of poisson sd and robust sd
        sd_eff = np.maximum(sd, sigma_rob.fillna(sd))
        g["lower"] = (lam - z * sd_eff).clip(lower=0)
        g["upper"] = lam + z * sd_eff
        g["rel_from_lam"] = (g["event_count"] - lam) / lam.replace(0, np.nan)
        g["anomaly_poisson_base"] = (
            (g["event_count"] >= min_count)
            & ((g["event_count"] < g["lower"]) | (g["event_count"] > g["upper"]))
            & (g["rel_from_lam"].abs() >= rel_min)
        )
        out.append(g[["event_name","event_date","event_count","lower","upper","rel_from_lam","anomaly_poisson_base"]])
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()

def pct_change_anoms(df, win=14, up=0.35, down=0.35, min_points=14, min_base=MIN_COUNT_DAY):
    """Simple % change vs rolling mean with volume threshold."""
    df = df.copy()
    df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")
    out = []
    for ev, g in df.groupby("event_name", sort=False):
        g = g.sort_values("event_date").copy()
        if len(g) < min_points:
            continue
        base = g["event_count"].rolling(win, min_periods=win//2).mean()
        pct = (g["event_count"] - base) / base.replace(0, np.nan)
        g["pct_change"] = pct.replace([np.inf, -np.inf], np.nan)
        g["anomaly_pct_base"] = (
            (base >= min_base)
            & ((g["pct_change"] > up) | (g["pct_change"] < -down))
        )
        out.append(g[["event_name","event_date","event_count","pct_change","anomaly_pct_base"]])
    return pd.concat(out, ignore_index=True) if out else pd.DataFrame()


import numpy as np
import pandas as pd

JOIN_KEYS = ["event_name","event_date","event_count"]

def _normalize_for_merge(df_in: pd.DataFrame) -> pd.DataFrame:
    if df_in is None or df_in.empty:
        return df_in
    df = df_in.copy()
    # ensure join key columns exist
    for c in JOIN_KEYS:
        if c not in df.columns:
            df[c] = np.nan
    # unify dtypes
    df["event_name"]  = df["event_name"].astype(str)
    df["event_date"]  = pd.to_datetime(df["event_date"], errors="coerce")\
                          .dt.tz_localize(None).dt.normalize()  # midnight, no tz
    df["event_count"] = pd.to_numeric(df["event_count"], errors="coerce").astype("Int64")
    return df

def safe_merge(left: pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
    if right is None or right.empty:
        return left
    L = _normalize_for_merge(left)
    R = _normalize_for_merge(right)
    return L.merge(R, on=JOIN_KEYS, how="left")

# -----------------------------
# Run detectors
# -----------------------------

r1 = robust_z_anoms(df, win=28, k=3.5)
r2 = weekday_robust_anoms(df, lookback_weeks=12, k=3.0)
r3 = poisson_band_anoms(df, win=28, z=2.5)
r4 = pct_change_anoms(df, win=14, up=0.35, down=0.35)

m = _normalize_for_merge(df[JOIN_KEYS].copy())
for x in [r1, r2, r3, r4]:
    m = safe_merge(m, x)


# fill NaNs
for c in m.columns:
    if c.startswith("anomaly_") and c.endswith("_base"):
        m[c] = m[c].fillna(False)

# optional persistence: require >= PERSIST_CONSEC consecutive anomaly days, same direction if available
def apply_persistence(group, base_cols, min_run=PERSIST_CONSEC):
    # choose a direction proxy (prefer robust_z, fallback to weekday_z or rel_from_lam)
    dir_series = group.get("robust_z")
    if dir_series is None or dir_series.isna().all():          # ✅ FIXED: added the missing dot
        dir_series = group.get("weekday_z")
    if dir_series is None or dir_series.isna().all():          # ✅ FIXED: added the missing dot here too
        dir_series = group.get("rel_from_lam")
    direction = np.sign(dir_series).fillna(0)

    base = group[base_cols].any(axis=1)

    # compute run-lengths of consecutive True with same sign
    run = np.zeros(len(base), dtype=int)
    last_sign = 0
    for i, (b, s) in enumerate(zip(base.values, direction.values)):
        if i == 0 or not b or s == 0 or s != last_sign:
            run[i] = 1 if b and s != 0 else 0
        else:
            run[i] = (run[i-1] + 1) if b else 0
        last_sign = s if b and s != 0 else 0
    persistent = run >= min_run if min_run > 1 else base.values
    group = group.copy()
    group["anomaly_any_base"] = base.values
    group["anomaly_any"] = persistent
    return group


base_cols = [c for c in m.columns if c.startswith("anomaly_") and c.endswith("_base")]
m = m.groupby("event_name", group_keys=False).apply(lambda g: apply_persistence(g.sort_values("event_date"), base_cols, PERSIST_CONSEC))

# cast booleans
flag_cols = ["anomaly_any_base","anomaly_any"] + base_cols
for c in flag_cols:
    m[c] = m[c].astype("boolean")

df_all_flags = m[m["anomaly_any"]].sort_values(["event_name","event_date"])

print("🚨 total anomalies (with persistence):", int(df_all_flags.shape[0]))
display_cols = ["event_name","event_date","event_count","robust_z","rel_delta","weekday_z","rel_from_lam","pct_change","anomaly_any"]
print(df_all_flags[display_cols].head(20))
